# Walkable Accessibility Score for Years 1997 to 2019

In [1]:
# Add this cell to time how long it takes to run the notebook

import timeit
start_time = timeit.default_timer()

In [2]:
import sys  
sys.path.insert(1, '/users/ifarah/appdata/roaming/python/python39/site-packages')

We must specify the correct directory path to load some packages. In the code below, REPLACE 'ifarah' with your personal username. It should be shown in the output directories of the installed packages above.

In [3]:
# Load libraries -- Takes 1.30 min approx
from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
import geopandas as gpd
from scipy import stats # for correlation
import gzip

In [4]:
# Only have to read once
# Read in the 2015 Block Group Shapefile for all the US.
s_v = gpd.read_file('../data/2015_US_BG/BG_mainland.shp') # Load geography (oftentimes as shapefile).

# Add 2011 GreatSchools school data
sch = gpd.read_file('../data/GreatSchools_2011_us48/GreatSchools_2011_us48.shp') 
sch = sch.to_crs('esri:102003')

#2021 ESRI parks data (centroids)
prk = gpd.read_file('../data/Centroids_for_USA_Parks_2021/parks.shp') 
prk = prk.to_crs('esri:102003')

# Change the Coordinate Reference System
s_v = s_v.set_crs('esri:102003', allow_override=True) # Set the Coordinate Reference System
s_v.rename(columns={'GEOID': 'ID'}, inplace=True) # Rename the columns for convenience

# Extract the centroids of the polygons.
# Replace the column "geometry" with the centroids of geography.
# This will change the geometry from "polygon" to "point" geometry.
s_v['geometry'] = s_v.centroid

s_v_filtered = s_v[['ID', 'geometry']]

## Create functions to run the walkable accessibility score

In [5]:
# This cell is creating a function for eastimating nearest neighbors from point to point.
def get_nearest_neighbors(gdf1, gdf2, k_neighbors=2):
    '''Find k nearest neighbors for all source points from a set of candidate points
    modified from: https://automating-gis-processes.github.io/site/notebooks/L3/nearest-neighbor-faster.html    
    Parameters
    ----------
    gdf1 : geopandas.DataFrame
    Geometries to search from.
    gdf2 : geopandas.DataFrame
    Geoemtries to be searched.
    k_neighbors : int, optional
    Number of nearest neighbors. The default is 2.
    Returns
    -------
    gdf_final : geopandas.DataFrame
    gdf1 with distance, index and all other columns from gdf2.'''

    src_points = [(x,y) for x,y in zip(gdf1.geometry.x , gdf1.geometry.y)]
    candidates =  [(x,y) for x,y in zip(gdf2.geometry.x , gdf2.geometry.y)]

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='euclidean')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    closest_gdfs = []
    for k in np.arange(k_neighbors):
        gdf_new = gdf2.iloc[indices[k]].reset_index()
        gdf_new['distance'] =  distances[k]
        gdf_new = gdf_new.add_suffix(f'_{k+1}')
        closest_gdfs.append(gdf_new)
    
    closest_gdfs.insert(0,gdf1)    
    gdf_final = pd.concat(closest_gdfs,axis=1)

    return gdf_final

def clean_dataframe(df):
    # Create the ID2 column
    df["ID2"] = df.index

    # Reshape the dataframe from wide to long format using the provided suffix
    long_df = pd.wide_to_long(df, stubnames=["distance_", "index_", "geometry_"], i="ID2", j="neighbor")

    # Rename columns
    long_df.loc[:, 'origin'] = long_df['ID']
    long_df.loc[:, 'dest'] = long_df['index_']
    long_df.loc[:, 'euclidean'] = long_df['distance_']

    # Reset index and keep necessary columns
    long_df = long_df.reset_index(level="neighbor")
    cost_df = long_df[['euclidean', 'origin', 'dest', 'neighbor']]

    # Sort the dataframe by origin and euclidean distance
    cost_df.sort_values(by=['origin', 'euclidean'], inplace=True)

    return cost_df

def access_measure(df_cost, df_sv, upper, decay):
    # Calculate time from euclidean distance
    # https://journals-sagepub-com.may.idm.oclc.org/doi/10.1177/0265813516641685
    df_cost['time'] = (df_cost['euclidean'] * 3600) / 5000  # convert distance into time (rate of 5kph)
    
    # Calculate LogitT_5 using the provided formula
    df_cost['LogitT_5'] = 1 - (1 / (np.exp((upper / 180) - decay * df_cost['time']) + 1))
    
    # Sum weighted distances by tract (origin) ID
    cost_sum = df_cost.groupby("origin").sum()
    cost_sum['ID'] = cost_sum.index
    
    # Merge with the corresponding smaller sv original dataframe
    cost_merge = df_sv.merge(cost_sum, how='inner', on='ID')
    
    return cost_merge


## Create the function to process the data from years 1997-2019

In [6]:
def process_business_data(year):
    # Load data for the specified year
    gz_file_path = f'../data/InfoUSA Data/{year}/{year}_Business_Academic_QCQ.txt.gz'
    
    # Open the gzipped file and read its content
    with gzip.open(gz_file_path, 'rt', encoding='latin-1') as f:
        df = pd.read_csv(f, sep=",", encoding='latin-1')

    # Amenities: groceries, restaurants, coffee shops, banks, parks, schools, bookstores, entertainment, and general shopping establishments 
    #schools (https://nces.ed.gov/programs/edge/geographic/schoollocations) and parks (centroids - https://www.arcgis.com/home/item.html?id=f092c20803a047cba81fbf1e30eff0b5)

    #Convert the column to string
    df['Primary NAICS Code'].astype(str)

    #Created new categories of NAICS codes so it was easier to filter the categories of interest.
    df['NAICS'] = df['Primary NAICS Code'].astype(str)
    df['NAICS2'] = df.NAICS.str[:2]
    df['NAICS4'] = df.NAICS.str[:4]
    df['NAICS6'] = df.NAICS.str[:6]
    df.NAICS4.value_counts()

    # Filter by specific amenity NAICS codes

    filtered = df.loc[(df['NAICS2'] == '72') | (df['NAICS4'] == '4421') | (df['NAICS4'] == '4431') | (df['NAICS4'] == '4451') | 
                    (df['NAICS4'] == '4461') | (df['NAICS4'] == '4481') | (df['NAICS4'] == '4482') | (df['NAICS4'] == '4483') |
                    (df['NAICS4'] == '4511') | (df['NAICS4'] == '4531') | (df['NAICS4'] == '4532') | (df['NAICS4'] == '4539') |
                    (df['NAICS4'] == '4453') | (df['NAICS4'] == '4523') | (df['NAICS4'] == '5221') | (df['NAICS6'] == '311811') |
                    (df['NAICS6'] == '451211')]

    # Remove Puerto Rico, Alaska, Hawaii, and US Virgin Islands because we will be measuring distances and islands will affect our analysis
    filtered = filtered[(filtered['State'] != 'PR') & (filtered['State'] != 'AK') & (filtered['State'] != 'HI') & (filtered['State'] != 'VI')]

    # Making sure that the latitude and longitude include all decimal points. # Is this right?
    filtered = filtered[filtered.Longitude != '-000.000-76']
    filtered = filtered[filtered.Latitude != '-000.000-76']

    # Create a geodataframe from coordinates (latitude and longitude)
    gdf = gpd.GeoDataFrame(
        filtered,
        geometry=gpd.points_from_xy(filtered.Longitude, filtered.Latitude),
        crs='epsg:4326') # epsg specifies the projection

        # Change the Coordinate Reference System (CRS)
    # Check for different projections here: https://epsg.io/
    gdf = gdf.to_crs('esri:102003')

    # Make sure that the geometry for each row has a value
    gdf = gdf[~gdf.is_empty]

    lst=[gdf,sch,prk]
    am=pd.concat(lst, ignore_index=True, axis=0)
    am["ID"] = am.index

    # Keep only the geometry column
    am_id = gdf[['geometry']]
    am_id

    #For 50 NN: #10 seconds
    closest50 = get_nearest_neighbors(s_v, am_id, k_neighbors=50)
    cost50 = clean_dataframe(closest50)
    result50_800 = access_measure(cost50, s_v_filtered, upper=800, decay=.008)


    result50_800['ID'] = result50_800['ID'].astype(str)
    
    # Save the final dataframe to a CSV with the year included in the file name
    output_file = f'result50_800_{year}.csv'
    output_file = f'../output/result50_800_{year}.csv'
    result50_800.to_csv(output_file, index=False)

In [7]:
import warnings
warnings.filterwarnings('ignore')

for year in range(1997, 2020):  # Adjust the range as needed
    process_business_data(year)

/var/folders/54/0j082m5x2mxcc2wcpp2_glsc0000gn/T/ipykernel_58641/2589850904.py:7: DtypeWarning: Columns (24,28,40,42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep=",", encoding='latin-1')
/Users/kevincredit/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/var/folders/54/0j082m5x2mxcc2wcpp2_glsc0000gn/T/ipykernel_58641/271410339.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cost_df.sort_values(by

## Merge CSV files into one file

In [8]:
#Merge .csv files
def merge_csv_files(csv_path_pattern, start_year, end_year, output_file):
    # Initialize an empty DataFrame to hold the merged data
    merged_data = None

    # Loop through each year
    for year in range(start_year, end_year + 1):
        # Generate the CSV file path for the current year
        csv_file = csv_path_pattern % year
        
        # Read the CSV file
        csv_data = pd.read_csv(csv_file, dtype={'ID': str})
        
        # Select the 'ID' and 'LogitT_5' columns and rename 'LogitT_5' to 'WAS{year}'
        csv_data = csv_data[['ID', 'LogitT_5']].rename(columns={'LogitT_5': f'WAS{year}'})
        
        # Merge the current CSV data with the previously merged data
        if merged_data is None:
            # For the first year, initialize the merged data
            merged_data = csv_data
        else:
            # Merge subsequent years on the 'ID' column
            merged_data = pd.merge(merged_data, csv_data, on='ID', how='outer')
    
    # Save the merged data to a CSV file
    merged_data.to_csv(output_file, index=False)


In [9]:
# Example usage
merge_csv_files(
  csv_path_pattern = "../output/result50_800_%d.csv",  # %d will be replaced by the year
  start_year = 1997,
  end_year = 2019,
    output_file="../output/merged_output.csv"
)

In [10]:
merged_data = pd.read_csv("../output/merged_output.csv", dtype={'ID': str})

In [11]:
merged_data.head(3)

,ID,WAS1997,WAS1998,WAS1999,WAS2000,WAS2001,WAS2002,WAS2003,WAS2004,WAS2005,...,WAS2010,WAS2011,WAS2012,WAS2013,WAS2014,WAS2015,WAS2016,WAS2017,WAS2018,WAS2019
0,010010201001,0.286106,0.232622,0.175505,0.213228,0.231888,0.245406,0.304709,0.259122,0.343507,...,1.029205,0.221847,0.209708,1.099580,1.148941,2.080988,2.068950,1.155888,1.114936,1.089932
1,010010201002,11.971912,10.375749,7.981109,8.779019,4.789293,8.798422,6.431561,8.018477,8.399062,...,4.809486,4.811480,5.613023,10.145084,7.630166,7.992792,5.596471,5.898272,2.705891,5.100125
2,010010202001,1.372614,1.241583,0.950506,0.763438,0.556322,0.961718,0.753916,1.743156,2.512532,...,0.467808,1.280566,1.342089,2.838298,2.719097,2.484229,2.288169,2.342801,2.057829,2.247687


## Merge CSV with block group shapefile (s_v)

In [12]:
final_shapefile_data = s_v_filtered.merge(merged_data, on='ID', how='left')

#Adding geometry column to the end
columns = [col for col in final_shapefile_data.columns if col != 'geometry'] + ['geometry']
final_shapefile_data = final_shapefile_data[columns]

In [13]:
final_shapefile_data.head(3)

,ID,WAS1997,WAS1998,WAS1999,WAS2000,WAS2001,WAS2002,WAS2003,WAS2004,WAS2005,...,WAS2011,WAS2012,WAS2013,WAS2014,WAS2015,WAS2016,WAS2017,WAS2018,WAS2019,geometry
0,060014001001,0.583724,0.355387,0.436912,0.388209,0.343679,0.348998,1.094018,1.140027,0.674874,...,1.327219,2.365213,2.593814,2.484240,1.686187,1.770627,1.965558,2.104157,1.501614,POINT (-2256868.242 354675.748)
1,060014002001,45.558356,45.508242,46.032436,45.879307,45.746954,45.860761,46.001192,46.065448,46.013966,...,45.916184,46.231157,46.261229,46.375803,46.251646,46.356328,46.268897,46.213793,46.196294,POINT (-2258832.974 353148.920)
2,060014002002,47.594279,47.550773,47.583356,47.725176,47.550251,47.621034,47.666230,47.715238,47.779969,...,47.538293,47.663359,47.572320,47.603393,47.351799,47.569731,47.627022,47.550301,47.554087,POINT (-2259050.925 352843.123)


In [14]:
len(final_shapefile_data)

215831

In [25]:
# Write shapefile to the output folder
final_shapefile_data.to_file(filename='../output/US_WAS_1997_2019.shp.zip',
                             driver='ESRI Shapefile',
                             compression='gzip')


In [16]:
elapsed = timeit.default_timer() - start_time
elapsed
# Approximately 129 minutes, a little bit more than 2 hours.

7738.431283209